In [12]:
import pandas as pd
import nltk,re
from nltk.tokenize import word_tokenize
from nltk import pos_tag, word_tokenize
from amr_logic_converter import types
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from word_forms.word_forms import get_word_forms
from sympy.logic.boolalg import to_cnf
from sympy.abc import A, B,C
import sympy
from sklearn.metrics import classification_report
from sympy import Symbol,simplify_logic
from pysat.formula import CNF
from pysat.solvers import Solver
from pattern.en import conjugate, lemma, lexeme, PRESENT, SG
import inflect
import numpy as np
from tqdm import tqdm
import spacy
nlp = spacy.load("en_core_web_sm")

In [13]:
from transition_amr_parser.parse import AMRParser
from amr_logic_converter import AmrLogicConverter
from transformers import pipeline
# Download and save a model named AMR3.0 to cache
parser = AMRParser.from_pretrained('AMR3-joint-ontowiki-seed43')
# tokens, positions = parser.tokenize('church-goers group sentences')
converter = AmrLogicConverter(existentially_quantify_instances=False,invert_relations=True)

load from cache /Users/xuyaofeng/.cache/torch/DATA/amr3joint_ontowiki2_g2g/models/amr3joint_ontowiki2_g2g-structured-bart-large/seed43/checkpoint_wiki.smatch_top5-avg.pt
| [en] dictionary: 157880 types
| [actions_nopos] dictionary: 58901 types
----------loading pretrained bart.large model ----------


Using cache found in /Users/xuyaofeng/.cache/torch/hub/pytorch_fairseq_main
2024-04-27 16:33:07 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/bart.large.tar.gz from cache at /Users/xuyaofeng/.cache/torch/pytorch_fairseq/40858f8de84f479771b2807266d806749e9ad0f8cb547921c35a76ae9c3ed0f6.099ef973524a5edb31b1211569b67bcc2863bc6d00781b79bac752acf8e48991
2024-04-27 16:33:09 | INFO | fairseq.tasks.denoising | dictionary: 50264 types


---------- task bart rewind: loading pretrained bart.large model ----------


Using cache found in /Users/xuyaofeng/.cache/torch/hub/pytorch_fairseq_main
2024-04-27 16:33:19 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/bart.large.tar.gz from cache at /Users/xuyaofeng/.cache/torch/pytorch_fairseq/40858f8de84f479771b2807266d806749e9ad0f8cb547921c35a76ae9c3ed0f6.099ef973524a5edb31b1211569b67bcc2863bc6d00781b79bac752acf8e48991
2024-04-27 16:33:20 | INFO | fairseq.tasks.denoising | dictionary: 50264 types


using CPU for models
pretrained_embed:  bart.large


Using cache found in /Users/xuyaofeng/.cache/torch/hub/pytorch_fairseq_main
2024-04-27 16:33:41 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/bart.large.tar.gz from cache at /Users/xuyaofeng/.cache/torch/pytorch_fairseq/40858f8de84f479771b2807266d806749e9ad0f8cb547921c35a76ae9c3ed0f6.099ef973524a5edb31b1211569b67bcc2863bc6d00781b79bac752acf8e48991
2024-04-27 16:33:42 | INFO | fairseq.tasks.denoising | dictionary: 50264 types


Using bart.large extraction in cpu (slow, wont OOM)
Finished loading models
self.machine_config:  /Users/xuyaofeng/.cache/torch/DATA/amr3joint_ontowiki2_g2g/models/amr3joint_ontowiki2_g2g-structured-bart-large/seed43/machine_config.json


In [14]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


2024-04-27 16:33:53 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
2024-04-27 16:33:54 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device: cpu


In [15]:
def generate_logic(data):
    tem  = []
    temm = []
    tem_token = []
    for sen in data:
        tokens, positions = parser.tokenize(sen)
        tem_token.append(tokens)
    
    annotations, machines = parser.parse_sentences(tem_token)
    tem = annotations
    temm = [i.get_amr().to_penman(jamr=False, isi=False) for i in machines]
    n = 0
    r1 = []
    r2 = []
    for sen in data:
        r1.append(converter.convert(tem[n]))
        r2.append(converter.convert(temm[n]))
        n+=1
    return tem,temm, r1,r2

In [16]:
def get_substring(s,w1,w2):
    p = inflect.engine()
#     print(s,w1,w2)
    if w1.isnumeric():
        w1 = p.number_to_words(w1)
    if w2.isnumeric():
        w2 = p.number_to_words(w2) 
    w11 = []
    if w1=="be-located-at":
        w11 = ["on","at","in"]
    w22 = []
    if w2=="be-located-at":
        w22 = ["on","at","in"]
    
    w111 = []
    if w1 == "person":
        w111 = [str(tok) for tok in nlp(s) if (tok.dep_ == "nsubj")]
    w222 = []
    if w2== "person":
        w222 = [str(tok) for tok in nlp(s) if (tok.dep_ == "nsubj")]


    sub1 = [j for i in get_word_forms(w1,0.7) for j in get_word_forms(w1,0.7)[i]]+[w1]+w11+w111
    sub2 = [j for i in get_word_forms(w2,0.7) for j in get_word_forms(w2,0.7)[i]]+[w2]+w22+w222
    search1 = 0
    search2 = 999
    c1= 0
    c2= 0

    token = word_tokenize(s.lower())
#     print(token)
    for i in range(len(token)):
        if token[i] in sub1:
            c1 = 1
            search1 = i
        elif token[i] in sub2:
            c2 = 1
            search2 = i
#     print(search1,search2)
# print(search)
    if c1 == 0 or c2 == 0:
#         rndtem = np.random.randint(len(token),size = 2)
         return False

    if search1 > search2:
#         print("The extracted string : " + " ".join(token[search2:search1+1]))
        return " ".join(token[search2:search1+1])
    else:
#         print("The extracted string : " + " ".join(token[search1:search2+1]))
        return " ".join(token[search1:search2+1])
        
 
# printing result


In [17]:
def combine(final,f = False):
    init = True
    for i in final:
        if type(i) == list:
            tem = True
            
            tem = tem&combine(i)
            if ~tem == -1:
                
                init = init & True
            elif ~tem == -2:
                if not f:
                    init = init & False
                else:
                    init = init & True
            else:
                init = init&~tem
        else:
            init = init&i
            
    return init
    

In [18]:
import copy
def transform(formula,Var,X):
    final = copy.deepcopy(formula)
    for i in range(len(final)):
        
        if type(final[i]) == list:
            
            if final[i][0] == "ARG":
                if " ".join([Var[final[i][1]],Var[final[i][2]],final[i][3]]) not in X:
                    continue
                else:
                    final[i] = X[" ".join([Var[final[i][1]],Var[final[i][2]],final[i][3]])]
#             
                
#                     init = init
            else:
                final[i] = transform(final[i],Var,X)
               
#                     else:
                    
        else:
            if final[i] not in X:
                continue
        
            else:
                final[i]  = X[final[i]] 
  
    return final
        


In [19]:
def extract(formula):
    and_list = []
    var = {}
    arg = []
    if type(formula) == types.Not:
#         print("ggg")
        return [extract(formula.body)[0]],{**extract(formula.body)[1],**var},arg+extract(formula.body)[2]
#     print(formula)
    for i in formula.args:
        if type(i) == types.Not:
            and_list.append(extract(i.body)[0])
            var = {**extract(i.body)[1],**var}
            arg = arg+extract(i.body)[2]
           
        else:
            if i.predicate.symbol[0] ==":":
#             if i.predicate.symbol[0] =="÷:"
                and_list.append(["ARG"]+ [i.terms[j].value for j in range(0,len(i.terms))]+[i.predicate.symbol])
                arg.append([i.terms[j].value for j in range(0,len(i.terms))]+[i.predicate.symbol])
            else:
                and_list.append(re.sub(r'\-*[0-9]',"",i.predicate.symbol))
                var[i.terms[0].value] = re.sub(r'\-*[0-9]',"",i.predicate.symbol)
    return and_list,var,arg
    

In [20]:
def score(s1,s2):
    sentences = [s1,s2]
    embedding_1= model.encode(sentences[0], convert_to_tensor=True,show_progress_bar=False)
    embedding_2 = model.encode(sentences[1], convert_to_tensor=True,show_progress_bar=False)
    return util.pytorch_cos_sim(embedding_1, embedding_2)[0][0]

In [21]:
def pysat_formula(formula):
    tem_list = []
    for i in str(formula).split(" & "):
        if i[0] == "x":
            tem_list.append([int(i[1:])])
        else:
            tem_tem = []
            for j in i.replace("(","").replace(")","").split(" | "):
                if j[0] == "~":
#                 print(int(j[-1]))
                    tem_tem.append(int(j[2:])*-1)
                elif j[0] == "x":
                    tem_tem.append(int(j[1:]))
            tem_list.append(tem_tem)
    return tem_list

In [22]:
def subsitute(x,y,replaceX,replaceXX,maxx,i,j,thre):
    tems = score(x,y)
                                    
    if tems>=thre:
        if tems > maxx[i]:
            maxx[i] = tems
            replaceXX[i] = replaceX[j]
            return True
    return False
            

In [37]:
def prove(data,sent):
    checkArg0 = []
    checkVaribale0 = {}
    for0,checkVaribale0,checkArg0 = extract(data[0])
    for i in checkArg0:
        for j in range(len(i)):
#             print(j)
            if i[j] in checkVaribale0:
                i[j] = checkVaribale0[i[j]]
            else:
                checkVaribale0[i[j]] = i[j]

    replaceX = {}
    n = 1
    for i in checkVaribale0:
        if checkVaribale0[i][0] == ":":
            continue
        if checkVaribale0[i] not in replaceX:
            replaceX[checkVaribale0[i]] = Symbol('x'+str(n))  
            n+=1
    for i in checkArg0:
#         print(i)
        if " ".join(i) not in replaceX:
            replaceX[" ".join(i)] = Symbol('x'+str(n))  
            n+=1
    explanation = False
    if len(data)>2:
        explanation = True
    if explanation:
        checkArg2 = []
        checkVaribale2 ={}
        for2,checkVaribale2,checkArg2 = extract(data[2])
        for i in checkArg2:
            for j in range(len(i)):
#             print(j)
                if i[j] in checkVaribale2:
                    i[j] = checkVaribale2[i[j]]
                else:
                    checkVaribale2[i[j]] = i[j]
    
#         print(checkVaribale2)
        for i in checkVaribale2:
#             print(i)
            if checkVaribale2[i][0] == ":":
                continue
            if checkVaribale2[i] not in replaceX:
                replaceX[checkVaribale2[i]] = Symbol('x'+str(n))  
                n+=1
        for i in checkArg2:
            if " ".join(i) not in replaceX:
                replaceX[" ".join(i)] = Symbol('x'+str(n))  
                n+=1
    checkArg11 = []
    checkVaribale11 ={}
    for1,checkVaribale11,checkArg11 = extract(data[1])
    
    quant = []
    for i in checkArg11:
        if i[-1][:6]==":quant":
            try:
                quant.append(checkVaribale11[i[1]])
            except:
                continue
    replaceXX = {}

    thre = 0.55
    for i in checkArg11:
        for j in range(len(i)):
            if i[j] in checkVaribale11:
                i[j] = checkVaribale11[i[j]]
            else:
#                 else:
                checkVaribale11[i[j]] = i[j]
    maxx = {}
    for i in checkArg11:
        maxx[" ".join(i)] = 0
    for i in checkVaribale11:
        maxx[checkVaribale11[i]] = 0
    temj = ""
    for i in checkVaribale11:
        if checkVaribale11[i] in quant:
            replaceXX[checkVaribale11[i]] = True
        if checkVaribale11[i][0] == ":":
            continue
        if checkVaribale11[i] in replaceX:
            replaceXX[checkVaribale11[i]] = replaceX[checkVaribale11[i]]

        else:
            ccccc = 0
            for j in replaceX:
                if len(j.split())>1:
                    continue

                subsitute(checkVaribale11[i],j,replaceX,replaceXX,maxx,checkVaribale11[i],j,thre)
                
            if maxx[checkVaribale11[i]] == 0:
                replaceXX[checkVaribale11[i]] =Symbol('x'+str(n))
#                 formula11 = &formula11
                
                n+=1
    for i in checkArg11:
        if " ".join(i) in replaceXX:
            continue
#         print(i)
        if " ".join(i) in replaceX:
#             print(i)
            replaceXX[" ".join(i)] = replaceX[" ".join(i)]

        else:
            temj = " "
            for j in replaceX:
                
#                 print(j)
                if len(j.split())<3:
#                     continue
                    if True:
#                         print(j,i,temj)
                        
                        if subsitute(j," ".join([i[0],i[-2]]),replaceX,replaceXX,maxx," ".join(i),j,thre):
                            temj = j
                               
                
                else:

                            tems3 = False
                            tems1 = get_substring(sent[1],i[0],i[-2])
                            tems2 = get_substring(sent[0],j.split()[0],j.split()[-2])
                            if explanation:
                                tems3 = get_substring(sent[2],j.split()[0],j.split()[-2])
#                            
                            if not tems1:
                                if tems2:
                                    if subsitute(" ".join([i[0],i[-2]]),tems2,replaceX,replaceXX,maxx," ".join(i),j,thre,):
                                        temj = j
    
                                if tems3:
                                    if subsitute(" ".join([i[0],i[-2]]),tems3,replaceX,replaceXX,maxx," ".join(i),j,thre):
                                        temj = j
                                   
                                if subsitute(" ".join([i[0],i[-2]])," ".join([j.split()[0],j.split()[1]]),replaceX,replaceXX,maxx," ".join(i),j,thre,):
                                    temj = j
                           
                                        
                            if tems2 and tems1:
                                if subsitute(tems1,tems2,replaceX,replaceXX,maxx," ".join(i),j,thre,):
                                    temj = j
                                
                        
                            if tems3 and tems1:
                                if subsitute(tems1,tems3,replaceX,replaceXX,maxx," ".join(i),j,thre,):
                                    temj = j
                   
            if maxx[" ".join(i)] == 0:
                replaceXX[" ".join(i)] = Symbol('x'+str(n))
                n+=1
            else:
#                 print(temj, i,maxx[" ".join(i)])
               
                if i[0] in replaceXX:
                    replaceXX[i[0]] = True
                if i[-2] in replaceXX:
                    replaceXX[i[-2]]= True
    new_rex = {}
    for i in replaceXX:
        if i == "and":
            new_rex[i] = True
        if i.split()[0]=="and" and i.split()[-1][:3]==":op":
            new_rex[i] = new_rex[i.split()[1]]
        else:
            new_rex[i] = replaceXX[i]
    new_re = {}
#     print(for2)
    for i in replaceX:
#         print(replaceXX.values())
        tcc = 0
        for j in new_rex:
        
            if isinstance(new_rex[j], sympy.Not):
                if ~new_rex[j] == replaceX[i]:
#                     print(new_rex[j] ,~new_rex[j] )
                    new_re[i] = replaceX[i]
                    tcc = 1
            else:
                if new_rex[j] == replaceX[i]:
                    new_re[i] = replaceX[i]
                    tcc = 1
        if tcc == 0:
                new_re[i] = True

    formula0 = combine(transform(for0,checkVaribale0, replaceX))

    formula11 =  combine(transform(for1,checkVaribale11, new_rex))
#     return 0
    if formula11 == -1:
        formula11 = True
    elif formula11 == -2:
        formula11 =False
    elif formula11 == 0:
        formula11 =False
    elif formula11 == 1:
        formula11 =True
    if explanation:
        formula2 = combine(transform(for2,checkVaribale2, replaceX))
        final_formula = to_cnf( (formula0 & formula2 ) & ~(formula11))
    else:
        final_formula = to_cnf( formula0 & ~(formula11))
    cnf = CNF(from_clauses=pysat_formula(final_formula))
   
    formula00 = combine(transform(for0,checkVaribale0, new_re))
    
    if explanation:
        formula22 = combine(transform(for2,checkVaribale2, new_re))
        if formula22 == -1:
            formula22 = True
        elif formula22 == -2:
            formula22 = False
        elif formula22 == 1:
            formula22 = True
        elif formula22 == 0:
            formula22 = False
        if formula00 == -1:
            formula00 = True
        elif formula00 == -2:
            formula00 =False
        elif formula00 == 1:
            formula00 =True
        elif formula00 == 0:
            formula00 =False
        final_formula11 = to_cnf( formula00 & formula22 & formula11)
    else:
        if formula00 == -1:
            formula00 = True
        elif formula00 == -2:
            fomula00 = False

        final_formula11 = to_cnf( formula00 & formula11)

    cnf11 = CNF(from_clauses=pysat_formula(final_formula11))
#
    with Solver(name = "Minisat22",bootstrap_with=cnf) as solver:
        
        check_ent = solver.solve()

    with Solver(name = "Minisat22",bootstrap_with=cnf11) as solver:
        check_con1 = solver.solve()
#     print(check_con)
    if not check_ent and check_con1:
        return "ent"
    elif not check_con1 and check_ent:

        return "con"
    
    elif not check_con1 and not check_ent:
        return "both"

    else:
        return "neu"
    
   

In [24]:
# np.random.seed(66)

In [25]:
df = pd.read_csv('esnli_train_1.csv')

In [26]:

sent_ent = []
sent_con = []
sent_neu = []
length = 2999999
for i in range(0,259999):

        try:
            if len(word_tokenize(df.iloc[i,2]))>length or len(word_tokenize(df.iloc[i,3]))>length or len(word_tokenize(df.iloc[i,4]))>length:
                continue
        except:
            continue

        if df.iloc[i,1] == "entailment":
#             conti

            sent_ent.append([df.iloc[i,2],df.iloc[i,3],df.iloc[i,4],'ent'])

        elif df.iloc[i,1] == "contradiction":
            sent_con.append([df.iloc[i,2],df.iloc[i,3],df.iloc[i,4],'con'])   
        else:
#             elif df.iloc[i,1] == "contradicton":
            sent_neu.append([df.iloc[i,2],df.iloc[i,3],df.iloc[i,4],'neu']) 


In [27]:
esnli_sent = [sent_ent[i] for i in np.random.choice(len(sent_ent), 50
                                                    ,replace=False)]+[sent_neu[i] for i in np.random.choice(len(sent_neu), 50,replace=False)]+[sent_con[i] for i in np.random.choice(len(sent_con), 50,replace=False)]

In [34]:
len(esnli_sent)

150

In [29]:
pre_data = []
for i in tqdm(esnli_sent):
#     try:
    pre_data.append(generate_logic(i[:-1])[-2])

  0%|                                                   | 0/150 [00:00<?, ?it/s]

Running on batch size: 3
3



decoding:   0%|                                           | 0/1 [00:00<?, ?it/s]/Users/xuyaofeng/Downloads/transition-amr-parser-master/src/fairseq_ext/sequence_generator_bartsv.py:1029: UserWarning: An output with one or more elements was resized since it had shape [3, 2], which does not match the required output shape [2, 2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Resize.cpp:24.)
  torch.add(
/Users/xuyaofeng/Downloads/transition-amr-parser-master/src/fairseq_ext/sequence_generator_bartsv.py:1040: UserWarning: An output with one or more elements was resized since it had shape [3, 1], which does not match the required output shape [2, 1]. This behavior is deprecated, and in a future PyTorch release outputs will not 

Running on batch size: 3
3



  1%|▌                                          | 2/150 [00:06<08:06,  3.29s/it]

...


 99%|████████████████████████████████████████▍| 148/150 [07:23<00:05,  2.79s/it]

Running on batch size: 3
3



 99%|████████████████████████████████████████▋| 149/150 [07:28<00:03,  3.30s/it]

Running on batch size: 3
3



100%|█████████████████████████████████████████| 150/150 [07:31<00:00,  3.01s/it]


In [38]:
ll = []
gl = []
for i in tqdm(range(0,150)):
    try:
#     pre_data = (generate_logic(i[:-1])[-2])
        tem = prove(pre_data[i][:],esnli_sent[i])
#         print(tem)
        if tem == "both":
            print("both")
            continue
        ll.append(tem)
        gl.append(esnli_sent[i][-1])
    except:
        print("exception")
        continue

    

 27%|███████████▏                              | 40/150 [00:35<01:56,  1.06s/it]

both


 41%|█████████████████                         | 61/150 [01:36<05:05,  3.43s/it]

both


 45%|██████████████████▊                       | 67/150 [01:51<03:35,  2.59s/it]

...

 80%|████████████████████████████████▊        | 120/150 [03:40<00:50,  1.68s/it]

both


 96%|███████████████████████████████████████▎ | 144/150 [04:13<00:11,  1.99s/it]

both


100%|█████████████████████████████████████████| 150/150 [04:22<00:00,  1.75s/it]


In [39]:
testy = gl
yhat_classes = ll

accuracy = accuracy_score(testy, yhat_classes)
print('Accuracy: %f' % accuracy)





print(classification_report(testy, yhat_classes))

Accuracy: 0.591241
              precision    recall  f1-score   support

         con       0.47      0.70      0.56        47
         ent       0.75      0.94      0.84        49
         neu       0.33      0.05      0.09        41

    accuracy                           0.59       137
   macro avg       0.52      0.56      0.50       137
weighted avg       0.53      0.59      0.52       137



In [40]:
confusion_matrix(testy, yhat_classes,labels = ["ent","con","neu"])

array([[46,  1,  2],
       [12, 33,  2],
       [ 3, 36,  2]])

In [41]:
from datasets import load_dataset

dataset = [load_dataset("sick",split="train"),load_dataset("sick",split="test"),load_dataset("sick",split="validation")]

In [42]:
sent_ent = []
sent_con = []
sent_neu = []
length = 10
# with open('sentences.txt', 'w') as f:
for j in dataset:
    for i in j:
#         print(i)÷
#         if df.iloc[i,1] != "entailment":
#             continue
        if len(word_tokenize(i["sentence_A"]))>length or len(word_tokenize(i["sentence_A"]))>length:
            continue
#         print(df.iloc[i,3])
        
#         print(generated_sentence)
        if i["label"] == 0:
            if i["entailment_AB"] == ' A_entails_B':
#             conti
                sent_ent.append([i["sentence_A"],i["sentence_B"],'ent'])
            else:
                sent_ent.append([i["sentence_B"],i["sentence_A"],'ent'])
        elif i["label"] == 1:
            if i["entailment_AB"] == ' A_neutral_B':
#             conti
                sent_neu.append([i["sentence_A"],i["sentence_B"],'neu'])
            else:
                sent_neu.append([i["sentence_B"],i["sentence_A"],'neu'])

        elif i["label"] == 2:
            if i["entailment_AB"] == ' A_contradicts_B':
#             conti
                sent_con.append([i["sentence_A"],i["sentence_B"],'con'])
            else:
                sent_con.append([i["sentence_B"],i["sentence_A"],'con'])


In [44]:

sick_sent = [sent_ent[i] for i in np.random.choice(len(sent_ent)
                                                   , 10,replace=False)]+[sent_neu[i] for i in np
    .random.choice(len(sent_neu), 10,replace=False)]+[sent_con[i] for i in np.random.choice(len(sent_con),10,replace = False)]




In [45]:
ll_sick = []
gl_sick = []
for i in tqdm(sick_sent):
    try:
        pre_data = (generate_logic(i[:-1])[-2])
        tem = prove(pre_data,i)
#         print(tem)
        if tem == "both":
            print("both")
            continue
        ll_sick.append(tem)
        gl_sick.append(i[-1])
    except:
        print("exception")
        continue

  0%|                                                    | 0/30 [00:00<?, ?it/s]

Running on batch size: 2
2



  3%|█▍                                          | 1/30 [00:01<00:45,  1.57s/it]

Running on batch size: 2
2



  7%|██▉                                         | 2/30 [00:03<00:57,  2.05s/it]

...


 93%|████████████████████████████████████████▏  | 28/30 [00:52<00:03,  1.89s/it]

Running on batch size: 2
2



 97%|█████████████████████████████████████████▌ | 29/30 [00:54<00:01,  1.71s/it]

Running on batch size: 2
2



100%|███████████████████████████████████████████| 30/30 [00:55<00:00,  1.86s/it]


In [46]:
testy = gl_sick
yhat_classes = ll_sick

accuracy = accuracy_score(testy, yhat_classes)
print('Accuracy: %f' % accuracy)

from sklearn.metrics import classification_report

print(classification_report(testy, yhat_classes))

Accuracy: 0.700000
              precision    recall  f1-score   support

         con       0.67      0.80      0.73        10
         ent       0.67      1.00      0.80        10
         neu       1.00      0.30      0.46        10

    accuracy                           0.70        30
   macro avg       0.78      0.70      0.66        30
weighted avg       0.78      0.70      0.66        30



In [ ]:
ll_sick


In [47]:
import json

data_mnli = []
with open('multinli_1.0_train.jsonl') as f:
    for line in f:
        data_mnli.append(json.loads(line))

In [48]:
sent_ent = []
sent_con = []
sent_neu = []
length = 10
# with open('sentences.txt', 'w') as f:
for i in data_mnli:
#         if df.iloc[i,1] != "entailment":
#             continue
        if len(word_tokenize(i["sentence1"]))>length or len(word_tokenize(i["sentence2"]))>length:
            continue
#         print(df.iloc[i,3])
        
#         print(generated_sentence)
        if i["gold_label"] == "entailment":
#             conti
            sent_ent.append([i["sentence1"],i["sentence2"],'ent'])

        elif i["gold_label"] == "contradiction":
            sent_con.append([i["sentence1"],i["sentence2"],'con'])   
        else:
#             elif df.iloc[i,1] == "contradicton":
            sent_neu.append([i["sentence1"],i["sentence2"],'neu']) 


In [49]:
mnli_sent = [sent_ent[i] for i in np.random.choice(len(sent_ent)
                                ,10,replace=False)]+[sent_neu[i] for i in np.random.choice(len(sent_neu), 10
                                                    ,replace=False)]+[sent_con[i] for i in np.random.choice(len(sent_con), 10
                                                    ,replace=False)]

In [50]:
ll_mn = []
gl_mn = []
for i in tqdm(mnli_sent):
    try:
        pre_data = (generate_logic(i[:-1])[-2])
        tem = prove(pre_data,i)
#         print(tem)
        if tem == "both":
            print("both")
            continue
        ll_mn.append(tem)
        gl_mn.append(i[-1])
    except:
        print("exception")
        continue

  0%|                                                    | 0/30 [00:00<?, ?it/s]

Running on batch size: 2
2



decoding:   0%|                                           | 0/1 [00:00<?, ?it/s]/Users/xuyaofeng/Downloads/transition-amr-parser-master/src/fairseq_ext/sequence_generator_bartsv.py:1029: UserWarning: An output with one or more elements was resized since it had shape [2, 2], which does not match the required output shape [1, 2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Resize.cpp:24.)
  torch.add(
/Users/xuyaofeng/Downloads/transition-amr-parser-master/src/fairseq_ext/sequence_generator_bartsv.py:1040: UserWarning: An output with one or more elements was resized since it had shape [2, 1], which does not match the required output shape [1, 1]. This behavior is deprecated, and in a future PyTorch release outputs will not 

Running on batch size: 2
2



  7%|██▉                                         | 2/30 [00:03<00:44,  1.60s/it]

...


 93%|████████████████████████████████████████▏  | 28/30 [01:12<00:05,  2.71s/it]

Running on batch size: 2
2



 97%|█████████████████████████████████████████▌ | 29/30 [01:15<00:02,  2.88s/it]

Running on batch size: 2
2



100%|███████████████████████████████████████████| 30/30 [01:17<00:00,  2.57s/it]


In [51]:
testy = gl_mn
yhat_classes = ll_mn

accuracy = accuracy_score(testy, yhat_classes)
print('Accuracy: %f' % accuracy)

from sklearn.metrics import classification_report

print(classification_report(testy, yhat_classes))

Accuracy: 0.500000
              precision    recall  f1-score   support

         con       1.00      0.30      0.46        10
         ent       0.50      0.70      0.58        10
         neu       0.38      0.50      0.43        10

    accuracy                           0.50        30
   macro avg       0.63      0.50      0.49        30
weighted avg       0.63      0.50      0.49        30

